A Tsetlin Machine to classify wheter a patient are in the class recurrence or non-recurrence


In [29]:
import random

In [30]:
breast_cancer_patients = [
    {'Menopause it40': False, 'Menopause ge40': True, 'Menopause premeno': False, 'Inv-nodes 0-2': False, 'Inv-nodes 3-5': True, 'Inv-nodes 6-8': False, 'Deg-Malig 1':False, 'Deg-Malig 2':False, 'Deg-Malig 3':True, 'Recurrence': True},
    {'Menopause it40': True, 'Menopause ge40': False, 'Menopause premeno': False, 'Inv-nodes 0-2': True, 'Inv-nodes 3-5': False, 'Inv-nodes 6-8': False, 'Deg-Malig 1':False, 'Deg-Malig 2':False, 'Deg-Malig 3':True, 'Recurrence': False},
    {'Menopause it40': False, 'Menopause ge40': True, 'Menopause premeno': False, 'Inv-nodes 0-2': False, 'Inv-nodes 3-5': False, 'Inv-nodes 6-8': True, 'Deg-Malig 1':False, 'Deg-Malig 2':False, 'Deg-Malig 3':True, 'Recurrence': True},
    {'Menopause it40': False, 'Menopause ge40': True, 'Menopause premeno': False, 'Inv-nodes 0-2': True, 'Inv-nodes 3-5': False, 'Inv-nodes 6-8': False, 'Deg-Malig 1':False, 'Deg-Malig 2':True, 'Deg-Malig 3':False, 'Recurrence': False},
    {'Menopause it40': False, 'Menopause ge40': False, 'Menopause premeno': True, 'Inv-nodes 0-2': True, 'Inv-nodes 3-5': False, 'Inv-nodes 6-8': False, 'Deg-Malig 1':False, 'Deg-Malig 2':False, 'Deg-Malig 3':True, 'Recurrence': True},
    {'Menopause it40': False, 'Menopause ge40': False, 'Menopause premeno': True, 'Inv-nodes 0-2': True, 'Inv-nodes 3-5': False, 'Inv-nodes 6-8': False, 'Deg-Malig 1':True, 'Deg-Malig 2':False, 'Deg-Malig 3':False, 'Recurrence': False},
]

In [31]:
#Rules for recurrence
R1 = ['Deg-Malig 3', 'NOT Menopause it40'] #recurrence rule
R2 = ['Deg-Malig 3'] #recurrence rule
R3 = ['Inv-node 0-2'] #non-recurrence rule

evaluate_condition() from Granmo's code 
    https://github.com/cair/TsetlinMachineBook/blob/main/Chapter_1.ipynb

In [32]:
def evaluate_condition(observation, condition):
    truth_value_of_condition = True
    for feature in observation:
        if feature in condition and observation[feature] == False:
            truth_value_of_condition = False
            break
        if 'NOT ' + feature in condition and observation[feature] == True:
            truth_value_of_condition = False
            break
    return truth_value_of_condition

In [33]:
evaluate_condition(breast_cancer_patients[0], R1)

True

Memory Class, also from Granmo's code

In [34]:
class Memory:
    def __init__(self, forget_value, memorize_value, memory):
        self.memory = memory
        self.forget_value = forget_value
        self.memorize_value = memorize_value
    
    def get_memory(self):
        return self.memory
    
    def get_literals(self):
        return list(self.memory.keys())
    
    def get_condition(self):
        condition = []
        for literal in self.memory:
            if self.memory[literal] >= 6:
                condition.append(literal)
        return condition
        
    def memorize(self, literal):
        if random() <= self.memorize_value and self.memory[literal] < 10:
            self.memory[literal] += 1
            
    def forget(self, literal):
        if random() <= self.forget_value and self.memory[literal] > 1:
            self.memory[literal] -= 1
            
    def memorize_always(self, literal):
        if  self.memory[literal] < 10:
            self.memory[literal] += 1

## Feedback

1. Recognize feedback (Type 1a feedback)
2. Erase feedback (Type 1b feedback)
3. Reject feedback

all are taken from Granmo's github repository

In [35]:
def type_i_feedback(observation, memory):
    remaining_literals = memory.get_literals()
    if evaluate_condition(observation, memory.get_condition()) == True: #If the condition is false, then the literals that are true for the object are remembered
        for feature in observation:
            if observation[feature] == True:
                memory.memorize(feature)
                remaining_literals.remove(feature)
            elif observation[feature] == False:
                memory.memorize('NOT ' + feature)
                remaining_literals.remove('NOT ' + feature)
    # else if the condition is false, all literals are forgotten
    for literal in remaining_literals:
        memory.forget(literal)

In [36]:
def type_ii_feedback(observation, memory):
    if evaluate_condition(observation, memory.get_condition()) == True: 
        for feature in observation:
            if observation[feature] == True: 
                memory.memorize_always(feature)
            elif observation[feature] == False: 
                memory.memorize_always('NOT ' + feature)

In [37]:
recurrence_rule = Memory(0.9, 0.1, {'Menopause it40': 5, 'NOT Menopause it40': 5,'Menopause ge40': 5, 'NOT Menopause ge40': 5,'Menopause premeno': 5, 
                                    'NOT Menopause premeno': 5, 'Inv-nodes 0-2': 5, 'NOT Inv-nodes 0-2': 5, 'Inv-nodes 3-5': 5, 'NOT Inv-nodes 3-5': 5, 
                                    'Inv-nodes 6-8': 5, 'NOT Inv-nodes 6-8': 5, 'Deg-Malig 1':5, 'NOT Deg-Malig 1':5, 'Deg-Malig 2':5, 'NOT Deg-Malig 2':5, 
                                    'Deg-Malig 3':5,'NOT Deg-Malig 3':5})

In [38]:
recurrence_rule.get_condition()

['NOT Menopause it40', 'Deg-Malig 3']